In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "notebooks"
    cd(joinpath(@__DIR__, "../"))
    Pkg.activate("Project.toml")
end

using MorphoMol

using CairoMakie
using LinearAlgebra
using Random
using Rotations
using StaticArrays
using Distributions
using Distances
using BenchmarkTools

  Activating project at `~/Desktop/Doktor/MorphoMol/MorphoMolMonteCarlo`


In [29]:
function state_to_poly(realization::Matrix, radii::Vector, filepath::String, n_same_color::Int)
    open(string(filepath, ".poly"), "w") do io
        println(io,"POINTS")
        color = ""
        for i in 1:size(realization, 2)
            if mod(i, n_same_color) == 1
                color = "c($(rand()),$(rand()),$(rand())"
            end
            println(io, "$(i): $(realization[1,i]) $(realization[2,i]) $(realization[3,i]) $(radii[i]) $(color), $(radii[i]))")
        end

        println(io,"POLYS")
        println(io,"END")
    end
end


function state_to_poly(x::Vector{Tuple{RotationVec{Float64}, SVector{3, Float64}}}, template_mol::Matrix{Float64}, radii::Vector, filepath::String)
    n_atoms_per_mol = size(template_mol)[2]
    n_mol = length(x)
    flat_realization = [(hvcat((n_mol), [R * template_mol .+ T for (R,T) in x]...)...)...]
    state_to_poly(flat_realization, radii, filepath, n_mol, n_atoms_per_mol)
end


function get_normal_distributed_translation(σ_t::Float64)
    return SVector{3, Float64}(randn(3)) * σ_t
end   

function get_normal_distributed_rotation(σ_r)
    exp(RotationVecGenerator((randn(3) * σ_r)...))
end


get_normal_distributed_rotation (generic function with 1 method)

In [30]:
function rotation_and_translation_gradient!(∇E,x,∇FSol,template_mol)
    for (i, (R,_)) in enumerate(x)
        ∇E[(i-1) * 6 + 1] = 0.5 * sum([-v[2]*(R[3,:] ⋅ w) + v[3]*(R[2,:] ⋅ w) for (v,w) in [(∇FSol[:,:,i][:,j], template_mol[:,j]) for j in 1:n_atoms_per_mol]])
        ∇E[(i-1) * 6 + 2] = 0.5 * sum([v[1]*(R[3,:] ⋅ w) - v[3]*(R[1,:] ⋅ w) for (v,w) in [(∇FSol[:,:,i][:,j], template_mol[:,j]) for j in 1:n_atoms_per_mol]])
        ∇E[(i-1) * 6 + 3] = 0.5 * sum([-v[1]*(R[2,:] ⋅ w) + v[2]*(R[1,:] ⋅ w) for (v,w) in [(∇FSol[:,:,i][:,j], template_mol[:,j]) for j in 1:n_atoms_per_mol]])
        ∇E[(i-1) * 6 + 4:(i-1) * 6 + 6] = sum([∇FSol[:,j,i] for j in 1:n_atoms_per_mol])
    end
    ∇E
end

function solvation_free_energy_gradient!(∇E, x, template_mol, radii, rs, pf, overlap_slope)
    n_atoms_per_mol = size(template_mol)[2]
    n_mol = length(x)
    flat_realization = [(hvcat((n_mol), [R * template_mol .+ T for (R,T) in x]...)...)...]
    _, dvol, dsurf, dmean, dgauss, dlol = MorphoMol.Energies.get_geometric_measures_and_overlap_value_with_derivatives(
        flat_realization,
        n_atoms_per_mol,
        radii,
        rs,
        0.0,
        overlap_slope
    )
    ∇FSol = reshape(pf[1] * dvol + pf[2] * dsurf + pf[3] * dmean + pf[4] * dgauss + dlol, (3, n_atoms_per_mol, n_mol))
    rotation_and_translation_gradient!(∇E, x, ∇FSol, template_mol)
end


solvation_free_energy_gradient! (generic function with 1 method)

In [31]:
function solvation_free_energy(x::Vector{Tuple{RotationVec{Float64}, SVector{3, Float64}}}, template_mol::Matrix{Float64}, radii::Vector{Float64}, rs::Float64, prefactors::AbstractVector, overlap_jump::Float64, overlap_slope::Float64, delaunay_eps::Float64)
    n_atoms_per_mol = size(template_mol)[2]
    n_mol = length(x)
    flat_realization = [(hvcat((n_mol), [R * template_mol .+ T for (R,T) in x]...)...)...]
    MorphoMol.Energies.solvation_free_energy(flat_realization, n_atoms_per_mol, radii, rs, prefactors, overlap_jump, overlap_slope, delaunay_eps)
end

solvation_free_energy (generic function with 1 method)

In [32]:
function position_leapfrog(ε, p, x)
    eps_p = ε * p
    for (i, (R,T)) in enumerate(x)
        si = (i-1) * 6
        x[i] = (exp(Rotations.RotationVecGenerator((eps_p[si + 1: si + 3])...)) * R, T + eps_p[si + 4: si + 6])
    end
    x
end

function leapfrog!(x, p, ∇E, β, ε, L, energy_gradient!)
    p -= ε * β / 2.0 * energy_gradient!(∇E, x)
    x = position_leapfrog(ε, p, x)
    for i in 1:L-1
        p -= ε * β * energy_gradient!(∇E, x)
        x = position_leapfrog(ε, p, x)
    end
    p -= ε * β / 2.0 * energy_gradient!(∇E, x)
    x, p
end

leapfrog! (generic function with 2 methods)

In [40]:
function simulate!(hmc::MorphoMol.Algorithms.HamiltonianMonteCarlo, x_init, p, iterations)
    energy, energy_gradient! = hmc.energy, hmc.energy_gradient!
    inner_product, draw_perturbation = hmc.inner_product, hmc.draw_perturbation
    leapfrog! = hmc.leapfrog!
    β, L, ε = hmc.β, hmc.L, hmc.ε

    x = deepcopy(x_init)
    x_backup = deepcopy(x)

    states = [deepcopy(x)]
    
    ∇E = zero(p)
    
    E = energy(x)
    
    accepted_steps = 0
    for _ in 1:iterations
        p = draw_perturbation()

        E_backup = E
        H_start = β*E + (1/2)*inner_product(p)

        x, p = leapfrog!(x, p, ∇E, β, ε, L, energy_gradient!)

        E = energy(x)
        H_end = β*E + (1/2)*inner_product(p)
        
        if rand() < min(1, exp(H_start - H_end)) 
            #accept step
            copyto!(x_backup, x)
            accepted_steps += 1
            push!(states, deepcopy(x))
        else
            # reject step
            E = E_backup
            copyto!(x, x_backup)
        end
    end
    states, accepted_steps
end

simulate! (generic function with 1 method)

In [38]:
x_init = [(exp(Rotations.RotationVecGenerator((rand(3) .* pi)...)), SVector{3, Float64}(rand(3)) * 10.0) for i in 1:12]
n_mol = length(x_init)

a = 1.6
template_mol = [0.0, 0.0, 0.0, 2*a, 0.0, 0.0, a, sqrt(3)*a, 0.0]
n_atoms_per_mol = length(template_mol) ÷ 3

template_mol = reshape(template_mol,(3,n_atoms_per_mol))
template_radii = [1.6, 1.6, 1.6]
radii = vcat([template_radii for i in 1:n_mol]...);

state_to_poly(x_init, template_mol, radii, "assets/output/start")

In [42]:
T = 5.0
ε = 0.01
L = 4
β = 1.0 / T

rs = 1.4
eta = 0.3665
pf = MorphoMol.Energies.get_prefactors(rs, eta)
overlap_slope = 10.0

σ_r = 0.25
σ_t = 0.5

p = zeros(6*n_mol)

draw_perturbation! = () -> vcat([[σ_r * randn(3); σ_t * randn(3)] for _ in 1:n_mol]...)
#TODO
inner_product = (p) -> (p ⋅ p)


energy(x) = solvation_free_energy(x, template_mol, radii, rs, pf, 0.0, overlap_slope, 1.0)
energy_gradient!(∇E, x) = solvation_free_energy_gradient!(∇E, x, template_mol, radii, rs, pf, overlap_slope)

hmc = MorphoMol.Algorithms.HamiltonianMonteCarlo(energy, energy_gradient!, inner_product, draw_perturbation!, leapfrog!, β, L, ε)
iterations = 100
states, accepted_steps = simulate!(hmc, deepcopy(x_init), p, iterations);
accepted_steps / iterations

0.76

In [35]:
for (i, state) in enumerate(states)
    state_to_poly(state, template_mol, radii, "assets/output/$(i)")
end